In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/covid19-radiography-database/COVID-19_Radiography_Dataset/Lung_Opacity.metadata.xlsx
/kaggle/input/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal.metadata.xlsx
/kaggle/input/covid19-radiography-database/COVID-19_Radiography_Dataset/README.md.txt
/kaggle/input/covid19-radiography-database/COVID-19_Radiography_Dataset/COVID.metadata.xlsx
/kaggle/input/covid19-radiography-database/COVID-19_Radiography_Dataset/Viral Pneumonia.metadata.xlsx
/kaggle/input/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal/Normal-859.png
/kaggle/input/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal/Normal-158.png
/kaggle/input/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal/Normal-10121.png
/kaggle/input/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal/Normal-1811.png
/kaggle/input/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal/Normal-97.png
/kaggle/input/covid19-radiography-database/COVID-19_Radi

# **Importing the necessary packages**

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from PIL import Image,ImageOps
import cv2 as cv
from tqdm import tqdm
from sklearn.utils import shuffle

# **Exploring the dataset**

In [3]:
normPath="/kaggle/input/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal/"
print("Number of normal images : ",len(os.listdir(normPath)))

Number of normal images :  10192


In [4]:
lunOpac="/kaggle/input/covid19-radiography-database/COVID-19_Radiography_Dataset/Lung_Opacity/"
print("Number of Lung opacity images : ",len(os.listdir(lunOpac)))

Number of Lung opacity images :  6012


In [5]:
virPneu="/kaggle/input/covid19-radiography-database/COVID-19_Radiography_Dataset/Viral Pneumonia/"
print("Number of viral pneumonia images : ",len(os.listdir(virPneu)))

Number of viral pneumonia images :  1345


In [6]:
covPath="/kaggle/input/covid19-radiography-database/COVID-19_Radiography_Dataset/COVID/"
print("Number of covid images : ",len(os.listdir(covPath)))

Number of covid images :  3616


# **Loading the images**

In [7]:
tot_images=[]

**Loading the covid images**

In [8]:
for image in tqdm(os.listdir(covPath)):
    img=Image.open(os.path.join(covPath,image))
    img=ImageOps.grayscale(img)
    img=img.resize((64,64))
    img=np.asarray(img)
    img=img.reshape((64,64,1))
    tot_images.append(img)

100%|██████████| 3616/3616 [00:27<00:00, 131.22it/s]


In [9]:
print(len(tot_images))

3616


**Loading the normal images**

In [10]:
for image in tqdm(os.listdir(normPath)):
    img=Image.open(os.path.join(normPath,image))
    img=ImageOps.grayscale(img)
    img=img.resize((64,64))
    img=np.asarray(img)
    img=img.reshape((64,64,1))
    tot_images.append(img)

100%|██████████| 10192/10192 [01:18<00:00, 129.77it/s]


In [11]:
print(len(tot_images))

13808


**Loading the Viral Pneumonia images**

In [12]:
for image in tqdm(os.listdir(virPneu)):
    img=Image.open(os.path.join(virPneu,image))
    img=ImageOps.grayscale(img)
    img=img.resize((64,64))
    img=np.asarray(img)
    img=img.reshape((64,64,1))
    tot_images.append(img)

100%|██████████| 1345/1345 [00:11<00:00, 122.00it/s]


In [13]:
print(len(tot_images))

15153


**Loading the lung opacity images**

In [14]:
for image in tqdm(os.listdir(lunOpac)):
    img=Image.open(os.path.join(lunOpac,image))
    img=ImageOps.grayscale(img)
    img=img.resize((64,64))
    img=np.asarray(img)
    img=img.reshape((64,64,1))
    tot_images.append(img)

100%|██████████| 6012/6012 [01:00<00:00, 98.91it/s]


In [15]:
print(len(tot_images))

21165


# **Marking down the labels for images**

In [16]:
img_labels=[1]*len(os.listdir(covPath))+[0]*(len(os.listdir(normPath))+len(os.listdir(virPneu))+len(os.listdir(lunOpac)))

In [17]:
print(len(img_labels))

21165


In [18]:
print(np.unique(img_labels,return_counts=True))

(array([0, 1]), array([17549,  3616]))


# **Converting the lists to arrays**

In [19]:
tot_images=np.array(tot_images)

In [20]:
img_labels=np.array(img_labels)

In [21]:
print(tot_images.shape,img_labels.shape)

(21165, 64, 64, 1) (21165,)


# **Normalising the dataset**

In [22]:
tot_images=tot_images/255

In [23]:
print(tot_images[0])

[[[0.22352941]
  [0.06666667]
  [0.03529412]
  ...
  [0.01568627]
  [0.01960784]
  [0.02352941]]

 [[0.21960784]
  [0.07058824]
  [0.03529412]
  ...
  [0.01568627]
  [0.01960784]
  [0.02352941]]

 [[0.20784314]
  [0.08627451]
  [0.1372549 ]
  ...
  [0.08627451]
  [0.04705882]
  [0.02352941]]

 ...

 [[0.14901961]
  [0.05490196]
  [0.04313725]
  ...
  [0.04313725]
  [0.04705882]
  [0.03921569]]

 [[0.18039216]
  [0.11764706]
  [0.11764706]
  ...
  [0.10588235]
  [0.10588235]
  [0.09803922]]

 [[0.25882353]
  [0.21176471]
  [0.21568627]
  ...
  [0.2       ]
  [0.2       ]
  [0.19607843]]]


# **Shuffling the dataset**

In [24]:
tot_images,img_labels=shuffle(tot_images,img_labels)

In [25]:
img_labels[1224]

0

# **Splitting the dataset into train and test**

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
x_train,x_test,y_train,y_test=train_test_split(tot_images,img_labels,test_size=0.2,stratify=img_labels)

In [28]:
print(x_test.shape,y_test.shape)

(4233, 64, 64, 1) (4233,)


In [29]:
print(x_train.shape,y_train.shape)

(16932, 64, 64, 1) (16932,)


In [30]:
print(np.unique(y_test,return_counts=True))

(array([0, 1]), array([3510,  723]))


In [31]:
print(np.unique(y_train,return_counts=True))

(array([0, 1]), array([14039,  2893]))


# **Balancing the train dataset**

In [32]:
print(np.unique(y_train,return_counts=True))

(array([0, 1]), array([14039,  2893]))


In [33]:
from imblearn.over_sampling import SMOTE

In [34]:
oversample = SMOTE(sampling_strategy=0.7)

In [35]:
x_train,y_train = oversample.fit_resample(x_train.reshape(x_train.shape[0],-1), y_train)

In [36]:
print(len(x_train),len(y_train))

23866 23866


In [37]:
print(np.unique(y_train,return_counts=True))

(array([0, 1]), array([14039,  9827]))


In [38]:
from imblearn.under_sampling import RandomUnderSampler

In [39]:
under = RandomUnderSampler(sampling_strategy=1)

In [40]:
x_train,y_train=under.fit_resample(x_train,y_train)

In [41]:
print(np.unique(y_train,return_counts=True))

(array([0, 1]), array([9827, 9827]))


In [42]:
x_train=x_train.reshape(x_train.shape[0],64,64,1)

In [43]:
print(x_train.shape,y_train.shape)

(19654, 64, 64, 1) (19654,)


# **Building the model**

# **Using VGG16 architecture**

In [44]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,BatchNormalization,Activation,Flatten,Dropout
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input 
from keras.applications import MobileNet
from keras.applications import VGG16
from keras.utils import to_categorical

In [45]:
input_tensor = Input(shape=(64,64,1))
vgg=VGG16(weights="imagenet",include_top=False)
x = Conv2D(3,(3,3),padding='same')(input_tensor)
vgg=vgg(x)
base_model = Model(inputs=input_tensor,outputs=vgg)

58892288/58889256 [==============================] - 1s 0us/step


In [46]:
base_model.trainable = False
for layer in base_model.layers:
    layer.trainable = False

In [47]:
model=Sequential()
model.add(base_model)
output=base_model.layers[-1].output
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [48]:
o=Adam(learning_rate=0.001)

In [49]:
model.compile(optimizer=o,loss="binary_crossentropy",metrics=['accuracy'])

In [50]:
model.fit(x_train,y_train,epochs=20,verbose=1)

Epoch 1/20
615/615 [==============================] - 186s 294ms/step - loss: 0.5052 - accuracy: 0.7441
Epoch 2/20
615/615 [==============================] - 180s 293ms/step - loss: 0.3936 - accuracy: 0.8253
Epoch 3/20
615/615 [==============================] - 180s 293ms/step - loss: 0.2772 - accuracy: 0.8825
Epoch 4/20
615/615 [==============================] - 180s 293ms/step - loss: 0.2518 - accuracy: 0.8964
Epoch 5/20
615/615 [==============================] - 180s 293ms/step - loss: 0.2595 - accuracy: 0.8899
Epoch 6/20
615/615 [==============================] - 180s 293ms/step - loss: 0.2294 - accuracy: 0.9051
Epoch 7/20
615/615 [==============================] - 180s 293ms/step - loss: 0.2222 - accuracy: 0.9055
Epoch 8/20
615/615 [==============================] - 180s 293ms/step - loss: 0.2123 - accuracy: 0.9126
Epoch 9/20
615/615 [==============================] - 180s 293ms/step - loss: 0.2024 - accuracy: 0.9160
Epoch 10/20
615/615 [==============================] - 180s 293m

In [51]:
y_hat=model.predict_classes(x_test)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


# **Evaluating the model**

In [52]:
from sklearn.metrics import accuracy_score,f1_score

In [53]:
print("Accuracy of the model : ",accuracy_score(y_test,y_hat))

Accuracy of the model :  0.9161351287502953


In [54]:
print("F1 score of the model : ",f1_score(y_test,y_hat))

F1 score of the model :  0.7740292807129217


In [55]:
from sklearn.metrics import confusion_matrix

In [56]:
confusion_matrix(y_test,y_hat)

array([[3270,  240],
       [ 115,  608]])

In [57]:
from sklearn.metrics import precision_score,recall_score

In [58]:
print(precision_score(y_test,y_hat))

0.7169811320754716


In [59]:
print(recall_score(y_test,y_hat))

0.8409405255878285


In [60]:
img=Image.open("/kaggle/input/covid19-radiography-database/COVID-19_Radiography_Dataset/Viral Pneumonia/Viral Pneumonia-228.png")
img=ImageOps.grayscale(img)
img=img.resize((64,64))
img=np.asarray(img)
img=img.reshape((64,64,1))
test=[]
test.append(img)

In [61]:
test=np.array(test)

In [62]:
test=test/255

In [63]:
yhat=model.predict_classes(test)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [64]:
yhat

array([[1]], dtype=int32)